In [1]:
%%capture

# Turn off warnings
# import warnings
# warnings.filterwarnings("ignore")

# Normal packages
import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import pandas as pd

# Format
from babel.numbers import format_currency

# Display
from IPython.display import HTML, Image, Markdown, display, display_html
from ipywidgets import AppLayout, Layout, interact
from shared_utils import portfolio_utils

# Settings
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:,.2f}".format

# GCS, del later since this will presumbly be read from a script that cleans up the data
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

# My utilities
import _utils

In [2]:
# DEL LATER
import altair as alt
import matplotlib.pyplot as plt
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [3]:
# Read in file
df_statewide = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

### CSIS Dataframe.

In [4]:
csis = df_statewide[
    [
        "project_name",
        "project_description",
        "mode_shift__csis_",
        "vmt__csis_",
        "dac_local_community_needs__csis_",
        "public_engagement__csis_",
        "safety__csis_",
        "zev__csis_",
        "climate_resiliency__csis_",
        "natural_resources_and_ecosystems__csis_",
        "infill_development_and_land_use__csis_",
    ]
]

In [5]:
csis = pd.melt(
    csis,
    id_vars=["project_name", "project_description"],
    value_vars=[
        "mode_shift__csis_",
        "vmt__csis_",
        "dac_local_community_needs__csis_",
        "public_engagement__csis_",
        "safety__csis_",
        "zev__csis_",
        "climate_resiliency__csis_",
        "natural_resources_and_ecosystems__csis_",
        "infill_development_and_land_use__csis_",
    ],
)

In [6]:
# Remove underscores off of old column names
csis["variable"] = csis["variable"].str.replace("_", " ").str.strip().str.title()
csis = csis.rename(columns={"variable": "csis", "value": "csis values"})

### Fake Benefit Dataframe.

In [7]:
benefit = df_statewide[
    [
        "project_name",
        "increase_peak_person_throughput",
        "reduction_in_peak_period_delay",
        "reduction_in_fatal_and_injury_crashes",
        "reduction_in_injury_rates",
        "increase_access_to_jobs",
        "increase_access_jobs_to_DAC",
        "commercial_dev_developed",
        "tons_of_goods_impacted",
        "improve_air_quality",
        "impact_natural_resources",
        "support_of_trasnportation",
    ]
]

In [8]:
benefit = pd.melt(
    benefit,
    id_vars=["project_name"],
    value_vars=[
        "increase_peak_person_throughput",
        "reduction_in_peak_period_delay",
        "reduction_in_fatal_and_injury_crashes",
        "reduction_in_injury_rates",
        "increase_access_to_jobs",
        "increase_access_jobs_to_DAC",
        "commercial_dev_developed",
        "tons_of_goods_impacted",
        "improve_air_quality",
        "impact_natural_resources",
        "support_of_trasnportation",
    ],
)

In [9]:
benefit["variable"] = benefit["variable"].str.replace("_", " ").str.strip().str.title()

In [10]:
benefit = benefit.rename(columns={"variable": "Benefit", "value": "Benefit Values"})

### Monetary Dataframe.

In [11]:
monetary_cols = [
    "project_name",
    "total_project_cost__$1,000_",
    "pa_ed_cost__$1,000_",
    "ps_e_cost__$1,000_",
    "row_cost__$1,000_",
    "con_cost__$1,000_",
    "non_infrastructure___plan_cost__$1,000_",
    "total_unfunded_need__$1,000_",
]

In [12]:
monetary = df_statewide[monetary_cols]

In [13]:
monetary = pd.melt(
    monetary,
    id_vars=["project_name"],
    value_vars=[
        "total_project_cost__$1,000_",
        "pa_ed_cost__$1,000_",
        "ps_e_cost__$1,000_",
        "row_cost__$1,000_",
        "con_cost__$1,000_",
        "non_infrastructure___plan_cost__$1,000_",
        "total_unfunded_need__$1,000_",
    ],
)

In [14]:
monetary["variable"] = (
    monetary["variable"].str.replace("_", " ").str.strip().str.title()
)

In [15]:
monetary = monetary.rename(
    columns={"variable": "Monetary Category", "value": "Monetary Values"}
)

In [16]:
monetary["Monetary Values $"] = monetary["Monetary Values"].apply(
    lambda x: format_currency(x, currency="USD", locale="en_US")
)

### Text table

In [17]:
text_cols = [
    "project_name",
    "full_county_name",
    "lead_agency",
    "district_full_name",
    "primary_mode",
    "secondary_mode_s_",
    "urban_rural",
    "current_phase",
    "urban_rural", 
    "route",
]

In [18]:
text_table = df_statewide[text_cols]

In [19]:
text_table2 = pd.melt(
    text_table,
    id_vars=["project_name"],
    value_vars=[
        "full_county_name",
        "lead_agency",
        "district_full_name",
        "primary_mode",
        "secondary_mode_s_",
        "urban_rural",
        "current_phase",
        "urban_rural",
        "route",
    ],
)

In [20]:
text_table2["variable"] = (
    text_table2["variable"].str.replace("_", " ").str.strip().str.title()
)

In [21]:
text_table2 = text_table2.rename(
    columns={"variable": "Basics", "value": "Basic Values"}
)

In [22]:
text_table2["Full Information"] = (
    text_table2["Basics"].astype("str")
    + ": "
    + text_table2["Basic Values"].astype("str")
)

In [23]:
text_table2['Zero'] = 0

### Ipywidgets 

* [Styling a widget](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#Description)
* [Altair and Widgets](https://medium.com/@chekos/altair-and-ipywidgets-a-simple-example-f61269c72c02)

In [26]:
my_list_of_projects = df_statewide.project_name.sort_values().unique().tolist()

In [27]:
test_widget = widgets.Dropdown(
    options=my_list_of_projects,
    value="Napa Valley Forward Sr 29 Up Valley Roundabout Project",
    description="Project:",
    disabled=False,
    style={"description_width": "initial"},
)

In [36]:
def all_charts(value):
    csis_chart = (
        alt.Chart(csis[csis["project_name"] == value])
        .mark_bar()
        .encode(
            x=alt.X(
                "csis values:Q",
                axis=None,
            ),
            y=alt.Y("csis:N", axis=alt.Axis(title="CSIS Category"), sort=("-x")),
            color=alt.Color(
                "csis", scale=alt.Scale(range=cp.CALITP_DIVERGING_COLORS), legend=None
            ),
            tooltip=["csis values", "csis"],
        )
        .properties(title=f"CSIS")
    )

    benefit_chart = (
        alt.Chart(benefit[benefit["project_name"] == value])
        .mark_bar()
        .encode(
            x=alt.X(
                "Benefit Values:Q",
                axis=None,
            ),
            y=alt.Y("Benefit:N", axis=alt.Axis(title="Benefit Category"), sort=("-x")),
            color=alt.Color(
                "Benefit",
                scale=alt.Scale(range=cp.CALITP_DIVERGING_COLORS),
                legend=None,
            ),
            tooltip=["Benefit", "Benefit Values"],
        )
        .properties(title="Benefits Information")
    )

    monetary_chart = (
        alt.Chart(monetary[monetary["project_name"] == value])
        .mark_bar()
        .encode(
            x=alt.X(
                "Monetary Values:Q",
                axis=None,
            ),
            y=alt.Y(
                "Monetary Category:N",
                axis=alt.Axis(title="Monetary Category"),
                sort=("-x"),
            ),
            color=alt.Color(
                "Monetary Category",
                scale=alt.Scale(range=cp.CALITP_DIVERGING_COLORS),
                legend=None,
            ),
            tooltip=["Monetary Category", "Monetary Values $"],
        )
        .properties(title="Monetary Information")
    )
    
    basic_chart = (
        alt.Chart(text_table2[text_table2["project_name"] == value])
            .mark_circle()
            .encode(x=alt.X("Zero:Q", axis=None), y=alt.Y("Full Information:N", axis=None))
            .properties(title=f"Overview of {value}"))
    
    basic_chart_text = (basic_chart.mark_text(
        align="center",
        baseline="middle",
        dx=3)
        .encode(text="Full Information:N")
           )
    
    
    csis_chart = _utils.preset_chart_config(csis_chart)
    benefit_chart = _utils.preset_chart_config(benefit_chart)
    monetary_chart = _utils.preset_chart_config(monetary_chart)
    basic_chart_text = _utils.preset_chart_config(basic_chart_text)
    
    charts =  alt.vconcat(basic_chart_text, monetary_chart,benefit_chart,csis_chart)
    return charts

In [37]:
interact(all_charts, value=test_widget)

interactive(children=(Dropdown(description='Project:', index=230, options=('#Raisefontana', '180 West Extensio…

<function __main__.all_charts(value)>